In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd "/content/drive/MyDrive/ColabNotebooks/yolov7"

/content/drive/MyDrive/ColabNotebooks/yolov7


In [ ]:
import os
if not os.path.exists('yolov7'):
    !git clone https://github.com/WongKinYiu/yolov7.git

%cd yolov7

!pip install -r requirements.txt

In [ ]:
# !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

In [1]:
%cd "/content/drive/MyDrive/ColabNotebooks/yolov7"
!python train.py --resume "./runs/train/yolov7_table_det/weights/best.pt"

/content/drive/MyDrive/ColabNotebooks/yolov7
2024-03-11 18:18:37.562820: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 18:18:37.562874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 18:18:37.563822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 18:18:37.569161: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-

In [ ]:
!python train.py --epochs 50 --workers 4 --device 0 --batch-size 16 \
--data data/table_det.yaml --img 640 640 --cfg cfg/training/yolov7_table_det.yaml \
--weights '' --name yolov7_table_det --hyp data/hyp.scratch.custom.yaml

2024-03-10 13:55:24.501600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 13:55:24.501654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 13:55:24.502994: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 13:55:24.510102: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 13:55:25.901937: W tensorflow/compiler/tf2

# Working with ICDAR2019 Dataset Data Cleaning

In [ ]:
import os
import shutil

def copy_files_from_text_file(text_file_path, source_folder, destination_folder):
    try:
        # Read names from the input text file
        with open(text_file_path, 'r') as text_file:
            file_names = [line.strip() for line in text_file.readlines()]

        # Iterate through each file name in the text file
        for file_name in file_names:
            # Handle the absence of file extension
            file_path_without_extension = os.path.join(source_folder, file_name)

            # Find the file with any extension in the source folder
            matching_files = [f for f in os.listdir(source_folder) if os.path.splitext(f)[0] == file_name]

            if matching_files:
                matching_file = matching_files[0]  # Take the first matching file
                file_path_with_extension = os.path.join(source_folder, matching_file)

                # Construct destination file path with extension
                destination_file_path = os.path.join(destination_folder, matching_file)

                # Copy the file from source to destination
                # shutil.copy(file_path_with_extension, destination_file_path)
                # print(f"File '{file_name}' copied to '{destination_folder}' successfully.")

                # Move the file from source to destination
                shutil.move(file_path_with_extension, destination_file_path)
                print(f"File '{file_name}' moved to '{destination_folder}' successfully.")
            else:
                print(f"File '{file_name}' not found in '{source_folder}'.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the input text file, source folder, and destination folder
source_folder = "/content/drive/MyDrive/ColabNotebooks/yolov7/dataset/master_labels"
destination_folder = "/content/drive/MyDrive/ColabNotebooks/yolov7/dataset/labels/train"
text_file_path = "/content/drive/MyDrive/ColabNotebooks/yolov7/dataset/master_list/test.txt"

# Call the function
copy_files_from_text_file(text_file_path, source_folder, destination_folder)

In [ ]:
import os

def list_files_and_write_to_text(folder_path, output_file):
    try:
        # Get a list of file names in the specified folder
        files = os.listdir(folder_path)

        # Open a text file for writing
        with open(output_file, 'w') as text_file:
            # Write each file name (without extension) to the text file and count the total number of files
            total_files = 0
            for file_name in files:
                # Exclude file extension
                file_name_without_extension, _ = os.path.splitext(file_name)
                text_file.write(file_name_without_extension + '\n')
                total_files += 1

        print(f"File names and total count written to {output_file} successfully.")
        print(f"Total number of files: {total_files}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the folder path and output text file name
folder_path = '/content/drive/MyDrive/ColabNotebooks/yolov3/input/data_root/dataset/JPEGImages'
output_file = '/content/drive/MyDrive/ColabNotebooks/yolov3/input/data_root/dataset/total.txt'

# Call the function
list_files_and_write_to_text(folder_path, output_file)


File names and total count written to /content/drive/MyDrive/ColabNotebooks/yolov3/input/data_root/dataset/total.txt successfully.
Total number of files: 1329


In [ ]:
import os
import random

def split_and_create_files(input_file, train_file, test_file, valid_file, train_percentage=80, test_percentage=10, valid_percentage=10):
    try:
        # Read file names from the input text file
        with open(input_file, 'r') as text_file:
            file_names = [line.strip() for line in text_file.readlines()]

        # Calculate the number of files for each split
        total_files = len(file_names)
        train_size = int(total_files * train_percentage / 100)
        test_size = int(total_files * test_percentage / 100)
        valid_size = int(total_files * valid_percentage / 100)

        # Shuffle the file names randomly
        random.shuffle(file_names)

        # Split the file names into train, test, and valid sets
        train_set = file_names[:train_size]
        test_set = file_names[train_size:(train_size + test_size)]
        valid_set = file_names[(train_size + test_size):]

        # Write file names to separate text files for train, test, and valid sets
        with open(train_file, 'w') as train_text_file:
            for file_name in train_set:
                train_text_file.write(file_name + '\n')
            print(f"Number of files in train set: {len(train_set)}")

        with open(test_file, 'w') as test_text_file:
            for file_name in test_set:
                test_text_file.write(file_name + '\n')
            print(f"Number of files in test set: {len(test_set)}")

        with open(valid_file, 'w') as valid_text_file:
            for file_name in valid_set:
                valid_text_file.write(file_name + '\n')
            print(f"Number of files in valid set: {len(valid_set)}")

        print("Files created successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the input file and output files for train, test, and valid sets
input_file = '/content/drive/MyDrive/ColabNotebooks/yolov3/input/data_root/dataset/total.txt'
train_file = '/content/drive/MyDrive/ColabNotebooks/yolov3/input/data_root/dataset/train.txt'
test_file = '/content/drive/MyDrive/ColabNotebooks/yolov3/input/data_root/dataset/test.txt'
valid_file = '/content/drive/MyDrive/ColabNotebooks/yolov3/input/data_root/dataset/valid.txt'

# Call the function
split_and_create_files(input_file, train_file, test_file, valid_file)


Number of files in train set: 1063
Number of files in test set: 132
Number of files in valid set: 134
Files created successfully.


In [ ]:
import random

def copy_random_names(input_file, output_file, num_names_to_copy=30):
    try:
        # Read names from the input text file
        with open(input_file, 'r') as input_text_file:
            all_names = [line.strip() for line in input_text_file.readlines()]

        # Ensure the requested number of names to copy is not more than the total available names
        num_names_to_copy = min(num_names_to_copy, len(all_names))

        # Create a copy of the list to avoid modifying the original list
        names_copy = list(all_names)

        # Randomly select 100 names from the copied list without duplicates
        random_names = random.sample(names_copy, num_names_to_copy)

        # Write the random names to the output text file
        with open(output_file, 'w') as output_text_file:
            for name in random_names:
                output_text_file.write(name + '\n')

        print(f"{num_names_to_copy} unique random names copied to {output_file} successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the input file and output file
input_file = '/content/drive/MyDrive/ColabNotebooks/yolov7/dataset/master_list/valid.txt'
output_file = '/content/drive/MyDrive/ColabNotebooks/yolov7/dataset/val.txt'

# Call the function
copy_random_names(input_file, output_file)


30 unique random names copied to /content/drive/MyDrive/ColabNotebooks/yolov7/dataset/val.txt successfully.


In [ ]:
!mv /content/drive/MyDrive/ColabNotebooks/yolov7/dataset/labels/test/* /content/drive/MyDrive/ColabNotebooks/yolov7/dataset/master_labels/

mv: cannot stat '/content/drive/MyDrive/ColabNotebooks/yolov7/dataset/labels/test/*': No such file or directory


In [ ]:
!ls '/content/drive/MyDrive/ColabNotebooks/yolov7/dataset/master_labels' -1 | wc -l
!ls '/content/drive/MyDrive/ColabNotebooks/yolov7/dataset/labels/test' -1 | wc -l

0
132
